In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
diabetes_dataset = pd.read_csv("Datasets/diabetes.csv")
diabetes_dataset

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
diabetes_dataset.shape

(768, 9)

In [4]:
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
diabetes_dataset['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [6]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [12]:
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

In [14]:
numerical_columns= ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
z_score = diabetes_dataset[numerical_columns].apply(zscore)

In [17]:
threshold = 3
outliers = (z_score.abs() > threshold).any(axis=1)
outliers

0      False
1      False
2      False
3      False
4       True
       ...  
763    False
764    False
765    False
766    False
767    False
Length: 768, dtype: bool

In [19]:
data_without_outliers = diabetes_dataset[~outliers]

In [21]:
scaler= StandardScaler()
scaled_features = scaler.fit_transform(data_without_outliers[numerical_columns])

In [23]:
scaled_data = pd.DataFrame(scaled_features, columns = [col + '_scaled' for col in numerical_columns])
categorical_columns = [col for col in diabetes_dataset.columns if col not in numerical_columns]
final_data = pd.concat([data_without_outliers[categorical_columns].reset_index(drop=True), scaled_data],axis =1)
final_data

,Outcome,Pregnancies_scaled,Glucose_scaled,BloodPressure_scaled,SkinThickness_scaled,Insulin_scaled,BMI_scaled,DiabetesPedigreeFunction_scaled,Age_scaled
0,1,0.657355,0.924040,-0.028115,0.923219,-0.805266,0.210285,0.606516,1.479220
1,0,-0.868490,-1.177082,-0.515765,0.533462,-0.805266,-0.848063,-0.364220,-0.183265
2,1,1.267694,2.091330,-0.678315,-1.350366,-0.805266,-1.346999,0.764788,-0.095766
3,0,-0.868490,-1.043678,-0.515765,0.143704,0.238698,-0.621274,-1.011378,-1.058257
4,0,0.352186,-0.143197,0.134435,-1.350366,-0.805266,-0.999256,-0.891795,-0.270764
...,...,...,...,...,...,...,...,...,...
683,0,1.878032,-0.643464,0.296984,1.767693,1.193814,0.104450,-0.997309,2.616709
684,0,-0.563321,0.056910,-0.190665,0.403542,-0.805266,0.694102,-0.402909,-0.533262
685,0,0.352186,0.023559,-0.028115,0.143704,0.438606,-0.908540,-0.737040,-0.270764
686,1,-0.868490,0.190315,-1.003415,-1.350366,-0.805266,-0.318889,-0.371255,1.216722


In [25]:
X = final_data.drop(columns = 'Outcome')
Y = final_data['Outcome']

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [29]:
print(X.shape,X_train.shape, X_test.shape)

(688, 8) (550, 8) (138, 8)


In [31]:
classifier = svm.SVC(kernel = 'linear')

In [33]:
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [35]:
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.7963636363636364


In [37]:
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.782608695652174


In [39]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [41]:
print("Training Accuracy:", model.score(X_train, Y_train))
print("Testing Accuracy:", model.score(X_test, Y_test))

Training Accuracy: 0.7981818181818182
Testing Accuracy: 0.7898550724637681


In [43]:
model = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
model.fit(X_train, Y_train)

LogisticRegression(solver='liblinear')

In [45]:
print("Training Accuracy:", model.score(X_train, Y_train))
print("Testing Accuracy:", model.score(X_test, Y_test))

Training Accuracy: 0.7981818181818182
Testing Accuracy: 0.7898550724637681


In [47]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
param_grid = {
    'C': [1,10,100,1000],
    'gamma': [0.0001,0.001,0.01,0.1],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2,3]
}

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
grid_search = GridSearchCV(SVC(), param_grid, cv=9, scoring='accuracy')
grid_search.fit(X_train_scaled, Y_train)

GridSearchCV(cv=9, estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000], 'degree': [2, 3],
                         'gamma': [0.0001, 0.001, 0.01, 0.1],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy')

In [53]:
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)
print("Train Accuracy:", best_model.score(X_train_scaled, Y_train))
print("Test Accuracy:", best_model.score(X_test_scaled, Y_test))

Best Parameters: {'C': 1, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
Train Accuracy: 0.8
Test Accuracy: 0.7898550724637681


In [55]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

base_estimators = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('svm', SVC(kernel='rbf', C=100, gamma=0.001))
]

model = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression())
model.fit(X_train_scaled, Y_train)
print("Train Accuracy:", model.score(X_train_scaled, Y_train))
print("Test Accuracy:", model.score(X_test_scaled, Y_test))

Train Accuracy: 0.8254545454545454
Test Accuracy: 0.7753623188405797


In [57]:
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    rf_param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1
)
rf_grid_search.fit(X_train_scaled, Y_train)
print("Best Random Forest Parameters:", rf_grid_search.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Random Forest Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


In [59]:
svm_param_grid = {
    'C': [1, 10, 100],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf']
}
svm_grid_search = GridSearchCV(
    SVC(),
    svm_param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1
)
svm_grid_search.fit(X_train_scaled, Y_train)
print("Best SVM Parameters:", svm_grid_search.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best SVM Parameters: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}


In [61]:
optimized_rf = rf_grid_search.best_estimator_
optimized_svm = svm_grid_search.best_estimator_

base_estimators = [
    ('rf', optimized_rf),
    ('svm', optimized_svm)
]

stacking_model = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression()
)

stacking_model.fit(X_train_scaled, Y_train)
print("Train Accuracy:", stacking_model.score(X_train_scaled, Y_train))
print("Test Accuracy:", stacking_model.score(X_test_scaled, Y_test))

Train Accuracy: 0.8145454545454546
Test Accuracy: 0.7753623188405797


In [63]:
input_data = (5, 160, 90, 35, 200, 33.0, 1.5, 50)
data_np_array = np.asarray(input_data)
input_data_reshaped = data_np_array.reshape(1,-1)
prediction = classifier.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[1]
The person is diabetic


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [65]:
import pickle

In [67]:
filename = 'diabetes_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [69]:
loaded_model = pickle.load(open('diabetes_model.sav', 'rb'))

In [71]:
for column in X.columns:
  print(column)

Pregnancies_scaled
Glucose_scaled
BloodPressure_scaled
SkinThickness_scaled
Insulin_scaled
BMI_scaled
DiabetesPedigreeFunction_scaled
Age_scaled
